## **Import library**

# **Build CNN for hand-written characters classification**

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

sns.set()

**Import dataset**

In [3]:
dataset = pd.read_csv("hand_written_dataset.csv").astype('float32')

**Preprocess the dataset**

In [4]:
dataset.shape

(372037, 785)

In [5]:
dataset['0']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
372032    25.0
372033    25.0
372034    25.0
372035    25.0
372036    25.0
Name: 0, Length: 372037, dtype: float32

In [6]:
dataset.rename(columns={'0':'label'}, inplace=True)
dataset['label']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
372032    25.0
372033    25.0
372034    25.0
372035    25.0
372036    25.0
Name: label, Length: 372037, dtype: float32

**Seperate label and dataset**

In [7]:
X = dataset.drop('label', axis=1)
y = dataset['label']
X.shape, y.shape

((372037, 784), (372037,))